DESCARGAR ARCHIVO, CONEXIÓN CON MySQL Y COMANDOS

In [ ]:
import pymysql
import requests
import io

In [ ]:
r = requests.get('https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ente-de-turismo/oferta-establecimientos-gastronomicos/oferta_gastronomica.csv')
#r.text
#r.encoding

df_datos = pd.read_csv(io.StringIO(r.text))
df_datos
lista_columnas = [tuple(df_locales[columnas].loc[fila]) for fila in range(0,len(df_locales))]


In [ ]:
conexion = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = 'contraseña',
    #port = '3306' Pero si es por defecto no debo declararlo.
    db = 'locales'
)

Traer registros

In [ ]:
cursor.execute('SELECT * FROM alumno')

alumnos = cursor.fetchall()
alumnos = cursor.fetchone() -->Trae el primero
alumnos = cursor.fetchmany(5) #-->Trae los primeros 5 registros

Insertar registros (hardcodeo)

In [ ]:
cursor.execute('INSERT INTO tabla(columnaA,columnaB) VALUES ('DatoA',DatoB)')
conexion.commit() #Es necesario hacerlo para subir los cambios a la base de datos.

In [ ]:
cursor = conexion.cursor()
cursor.executemany('INSERT INTO locales(Nombre,Categoria,Direccion,Barrio,Comuna) VALUES (%s,%s,%s,%s,%s)', (lista_columnas))
conexion.commit()

Insertar registros (desde variables)

In [ ]:
nombre = 'Adalberto'
edad = 30
cursor.executemany('INSERT INTO tabla(columnaA,columnaB) VALUES (%s,%s)', (nombre,edad))
conexion.commit()

Insertar registros (mayor cantidad)

In [ ]:
personas = [
    ('Pablo',22),
    ('Roberta',25),
    ('Saul',41),]
cursor.executemany('INSERT INTO tabla(columnaA,columnaB) VALUES (%s,%s)', personas)
conexion.commit()

Para cerrar la conexión al salir

In [ ]:
conexion.close()

UNA FORMA DE DESCARGAR Y PREPARAR LOS DATOS PARA SUBIR A MYSQL (EFICIENTE Y SIN USAR PANDAS)

In [ ]:
from typing import IO
import csv

descarga = requests.get('URL')              #descarga del archivo CSV de la web.

descarga.encoding= 'utf-8'                  #cambiamos la codificación (tildes, caracteres, etc.).

archivo = csv.reader(StringIO(descarga))    #abrimos lo descargado con la librería csv.

next(archivo)                               #saltamos la primera línea (tiene los nombres de columna).

for fila in archivo:                        #exploramos el archivo.
    print(fila)

datos = [(linea[3], linea[5],linea[6],linea[10],linea[15][7:]) for linea in archivo]    #armamos las tuplas para importar en MySQL.
                                                        #--> línea 15 desde el campo 7 en adelante.
 
#Procedemos con la conexión y subida del archivo.                                                                                                 

OTRA FORMA DE DESCARGAR Y PREPARAR LOS DATOS (USANDO PANDAS)

In [ ]:
import pymysql
import requests
import pandas as pd
import io
import numpy as np

descarga = requests.get('URL')                      #descarga del archivo CSV de la web.

descarga.encoding= 'utf-8'                  #cambiamos la codificación (tildes, caracteres, etc.).

df_datos = pd.read_csv(io.StringIO(descarga.text))  #creamos un DataFrame a partir del archivo descargado.

columnas = ['nombre','categoria','direccion_completa','barrio','comuna']    #Armo lista con las columnas que voy a usar.

df_locales = df_datos[columnas]                     #creamos un nuevo DataFrame con las columnas necesarias.

df_locales[columnas].isna().sum()                   #verificar si hay valores nulos.
df_locales.replace(np.nan, None, inplace=True)      #cambiar los valores nulos por None (MySQL no lee los NaN)

lista_columnas = [tuple(df_locales[columnas].loc[fila]) for fila in range(0,len(df_locales))]   #creamos el listado de tuplas.

#Procedemos con la conexión y subida del archivo. 